In [5]:
import requests
import psycopg2
from psycopg2 import sql
import os
from dotenv import load_dotenv
import pandas as pd
from Bitcoin_api import fetch_bitcoin_data
from scrappy import get_bitcoin_news

          date                                           headline  \
0   2023-12-04  Bitcoin price surges above $42,000 as rate cut...   
1   2023-12-05                       Bitcoin’s bounceback déjà vu   
2   2023-12-05  Buying frenzy puts some Grayscale crypto funds...   
3   2023-12-05     The jobs market is still tight Premium content   
4   2023-12-07                               The return of crypto   
..         ...                                                ...   
100 2024-05-22           First UK crypto ETPs to launch on May 28   
101 2024-05-23  SEC paves way for ethereum ETFs in boost for c...   
102 2024-05-24      Cryptofinance: into the ether Premium content   
103 2024-05-24  British-Chinese bitcoin money launderer jailed...   
104 2024-05-30     European bitcoin ETPs suffer mounting outflows   

                                           subheadline  
0    Gold touches all-time high as speculation grow...  
1    The buy-and-hold case for crypto remains large...  


## Connecting to DataBase and Creating Tables

In [2]:
# make sure to load dotenv
load_dotenv()

# Database connection paramete(s
Database_URL = os.getenv('DataBase_Url')

#connect to database
conn = psycopg2.connect(Database_URL)
cur = conn.cursor()

#create tables if they don't exist

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_Prices(
    Date DATE PRIMARY KEY,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Volume FLOAT
    
)            
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_News(
    id SERIAL PRIMARY KEY,
    date DATE,
    headline TEXT,
    subheadline TEXT
    
)            
''')

conn.commit()

Python-dotenv could not parse statement starting at line 11


## Insertion of Bitcoin Prices to Table

In [3]:
#function to insert Bitcoin_Prices
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_Prices(Date, Open, High, Low, Close, Volume)
                Values(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (Date) DO NOTHING
                ''',(index, row['1. Open'], row['2. High'], row['3. Low'], row['4. Close'], row['5. Volume']))
        conn.commit()
        cursor.close()
        print("success")
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Insertion of  Bitcoin News to Table

In [6]:
#function to insert Bitcoin_News
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_News (id, date , headline, subheadline)
                Values(%s, %s, %s, %s)
                ON CONFLICT (id) DO NOTHING
                ''',(index,row['date'], row['headline'], row['subheadline']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Fetching and Inserting Data

In [7]:
#Main Function to orchestrate the data fetching and insertion
def main():
    # fetch Bitcoin Data
    bitcoin_data = fetch_bitcoin_data("BTC","USD")
    #Scrape Bitcoin News data
    bitcoin_headlines_df = get_bitcoin_news(last_page=72)
    #connect to postgresSQL database
    conn = psycopg2.connect(Database_URL)

    if conn:
        #Insert data into database
        insert_bitcoin_data(conn,bitcoin_data)
        insert_bitcoin_news(conn,bitcoin_headlines_df)

        #close the database connection
        conn.close()
    else:
        print('Failed to connect to the database')

if __name__== '__main__':
    main()

new one

# drop

In [8]:
import os
import psycopg2

def drop_tables(table_names, conn):
    """
    Drops the specified tables from the PostgreSQL database.

    Args:
    table_names (list of str): List of table names to be dropped.
    conn: The psycopg2 connection object to the PostgreSQL database.
    """
    # Create a cursor object
    cur = conn.cursor()

    # Iterate over the list of table names and drop each one
    for table_name in table_names:
        drop_table_query = f'DROP TABLE IF EXISTS {table_name} CASCADE'
        cur.execute(drop_table_query)
        print(f'Table {table_name} dropped.')

    # Commit the changes and close the cursor
    conn.commit()
    cur.close()

if __name__ == "__main__":
    # List of table names to be dropped
    table_names = [
        'bitcoin_news_update',
        'bitcoin_news_u',
        
    ]

    # Connect to the PostgreSQL database using the DATABASE_URL environment variable
    DATABASE_URL = os.getenv('DATABASE_URL')
    conn = psycopg2.connect(DATABASE_URL)

    try:
        # Call the function to drop the tables
        drop_tables(table_names, conn)
    finally:
        if conn:
            conn.close()


Table bitcoin_news_update dropped.
Table bitcoin_news_u dropped.


### Trail"""

In [39]:
import os
import psycopg2
from dotenv import load_dotenv

# Function to create the BTC_DATA_OK table if it doesn't exist
def create_table(conn):
    try:
        cur = conn.cursor()
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS BTC_DATA_OK (
            Date DATE PRIMARY KEY,
            Open FLOAT,
            High FLOAT,
            Low FLOAT,
            Close FLOAT,
            Volume FLOAT
        )
        '''
        cur.execute(create_table_query)
        conn.commit()
        print("Table created successfully")
    except Exception as e:
        print(f"Error creating table: {e}")
    finally:
        cur.close()

# Function to insert data into the PostgreSQL database
def insert_data_into_db(conn, df):
    try:
        cur = conn.cursor()

        for index, row in df.iterrows():
            insert_query = '''
            INSERT INTO BTC_DATA_OK (Date, Open, High, Low, Close, Volume) 
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (Date) DO NOTHING
            '''
            values = (index, row["1. Open"], row["2. High"], row["3. Low"], row["4. Close"], row["5. Volume"])
            cur.execute(insert_query, values)
        
        conn.commit()
        print("Data inserted successfully")
    except Exception as e:
        print(f"Error inserting data: {e}")
    finally:
        cur.close()

def main():
    # Fetch the DATABASE_URL from environment variable
    DATABASE_URL = os.getenv('DataBase_Url')
    # Ensure DATABASE_URL is retrieved correctly
    if not DATABASE_URL:
        raise ValueError("DATABASE_URL environment variable is not set.")

    # Load environment variables from .env file
    load_dotenv()
    # Fetch the API key from environment variable
    API_KEY = os.getenv('AlphaVantage_API_key')

    # Ensure API_KEY is retrieved correctly
    if not API_KEY:
        raise ValueError("API_KEY environment variable is not set.")

    # Fetch Bitcoin data using the imported function
    df = fetch_bitcoin_data(API_KEY, "BTC", "USD")

    # Check if the DataFrame is valid
    if df is None or df.empty:
        raise ValueError("Data fetching returned None or an empty DataFrame. Please check the fetch_bitcoin_data function.")

    print(f"Fetched DataFrame:\n{df.head()}\n[Total Rows: {df.shape[0]}]")

    # Connect to the PostgreSQL database
    conn = psycopg2.connect(DATABASE_URL)

    # Create the table
    create_table(conn)

    # Insert data into the database
    insert_data_into_db(conn, df)

    # Close the connection
    conn.close()

if __name__ == "__main__":
    main()


ValueError: API_KEY environment variable is not set.